First import the requisite modules.

In [ ]:
"""GridRad Severe demo/test."""

%load_ext autoreload
%autoreload 2

from pathlib import Path
import shutil
import numpy as np
import thuner.data as data
import thuner.option as option
import thuner.analyze as analyze
import thuner.parallel as parallel
import thuner.visualize as visualize
import thuner.track as track
import thuner.default as default

notebook_name = "gridrad_demo.ipynb"

Next, specify the folders where THUNER outputs will be saved.

In [ ]:
# Parent directory for saving outputs
base_local = Path.home() / "THUNER_output"
output_parent = base_local / f"runs/gridrad/gridrad_demo"
options_directory = output_parent / "options"
visualize_directory = output_parent / "visualize"

# Delete the options directory if it already exists
if output_parent.exists():
    shutil.rmtree(output_parent)

Now specify the options for the THUNER run, beginning with the GridRad severe dataset 
options. Options classes in THUNER are built on the `pydantic` `BaseModel`, and are
initialized by passing keyword, value pairs. Note that GridRad severe data is organized 
into "events". First load the example events from local disk. If you don't yet have the 
demo data, run thuner.data.get_demo_data().

In [ ]:
event_directories = data.gridrad.get_event_directories(year=2010, base_local=base_local)
event_directory = event_directories[0] # Take the first event from 2010 for the demo
# Get the start and end times of the event, and the date of the event start
start, end, event_start = data.gridrad.get_event_times(event_directory)
start = "2010-01-20T23:00"
end = "2010-01-21T01:00"
times_dict = {"start": start, "end": end}
gridrad_dict = {"event_start": event_start}
gridrad_options = data.gridrad.GridRadSevereOptions(**times_dict, **gridrad_dict)

All instances of options classes can be examined using the `model_dump` method, which 
converts the class instance to a dictionary. Note the `parent_local` field, which provides
the parent directory on local disk containing the dataset. Analogously, `parent_remote` 
specifies the remote location of the data. Note also the `filepaths` field, which
provides a list of the dataset's absolute filepaths. If `filepaths` is unset
when the options instance is created, it will be populated automatically based on
the other input arguments. Note the `use` field, which tells THUNER whether the dataset 
will be used to `track` or `tag` objects.

In [ ]:
gridrad_options.model_dump()

We will use ERA5 single-level and pressure-level data for tagging the storms detected 
in the GridRad Severe dataset with other attributes, e.g. ambient winds.

In [ ]:
era5_dict = {"latitude_range": [27, 39], "longitude_range": [-102, -89]}
era5_pl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.era5.ERA5Options(**times_dict, **era5_dict)

All the dataset options are grouped into a single `DataOptions` object, which is passed to the THUNER tracking function. We also save these options as a YAML file.

In [ ]:
datasets = [gridrad_options, era5_pl_options, era5_sl_options]
data_options = option.data.DataOptions(datasets=datasets)
data_options.to_yaml(options_directory / "data.yml")

Now create and save options describing the grid. If `regrid` is `False` and grid 
properties like `altitude_spacing` or `geographic_spacing` are set to `None`, THUNER
will attempt to infer these from the tracking dataset.

In [ ]:
# Create and save the grid_options dictionary
kwargs = {"name": "geographic", "regrid": False, "altitude_spacing": None}
kwargs.update({"geographic_spacing": None})
grid_options = option.grid.GridOptions(**kwargs)
grid_options.to_yaml(options_directory / "grid.yml")

Finally, we create options describing how the tracking should be performed. In 
multi-feature tracking, some objects, like mesoscale convective systems (MCSs), can be defined in terms of others, like convective and stratiform echoes. THUNER's approach is to first specify object options seperately for each object type, e.g. convective echoes, stratiform echoes, mesoscale convective systems, and so forth. Object options are specified using `pydantic` models which inherit from `BaseObjectOptions`. Related objects are then grouped together into `LevelOptions` models. The final `TrackOptions` model, which is passed to the tracking function, then contains a list of `LevelOptions` models. The idea is that "lower level" objects, can comprise the building blocks of "higher level" objects, with THUNER processing the former before the latter.

In this tutorial, level 0 objects are the convective, middle and stratiform echo regions, 
and level 1 objects are mesoscale convective systems defined by grouping the level 0 objects. Because `TrackOptions` models can be complex to construct, a function for 
creating a default `TrackOptions` model matching the approach of [Short et al. (2023)](https://doi.org/10.1175/MWR-D-22-0146.1) is defined in the module `default.track`.

In [ ]:
# Create the track_options dictionary
track_options = default.track(dataset="gridrad")
# Show the options for the level 0 objects
print(f"Level 0 objects list: {track_options.levels[0].object_names}")
# Show the options for the level 1 objects
print(f"Level 1 objects list: {track_options.levels[1].object_names}")

Note a core component of the options for each object is the ``atributes`` field, which describes how object attributes like position, velocity and area, are to be retrieved and stored. See the attributes tutorial for a detailed explanation of how THUNER handles attributes. To illustrate, below we print the options which describe how latitude and longitude coordinates are retrieved for MCS objects.

In [ ]:
# Show the options for mcs coordinate attributes
mcs_attributes = track_options.object_lookup["mcs"].attributes
core_mcs_attributes = mcs_attributes.attribute_type_lookup["core"]
core_mcs_attributes.attribute_lookup["coordinates"].model_dump()

The default `TrackOptions` use "local" and "global" cross-correlations to measure object velocities, as described by [Raut et al. (2021)](https://doi.org/10.1175/JAMC-D-20-0119.1) and [Short et al. (2023)](https://doi.org/10.1175/MWR-D-22-0146.1). For GridRad severe, we modify this approach slightly so that "global" cross-correlations are calculated using boxes encompassing each object, with a margin of 70 km around the object.

In [ ]:
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.unique_global_flow = False
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.to_yaml(options_directory / "track.yml")

Users can also specify visualization options for generating figures during a tracking run.
Uncomment the line below to generate figures that visualize the matching algorithm - naturally 
this makes a tracking run much slower.

In [ ]:
visualize_options = None
# visualize_options = default.runtime(visualize_directory=visualize_directory)
# visualize_options.to_yaml(options_directory / "visualize.yml")

Now that all the options for the tracking run have been specified, we can pass these to
the tracking function. We also need to generate the times used for the tracking run.

In [ ]:
analyze.utils.read_options(output_parent / "interval_0")

In [ ]:
times = data._utils.generate_times(data_options.dataset_by_name("gridrad"))
args = [times, data_options, grid_options, track_options, visualize_options]
num_processes = 4 # If visualize_options is not None, num_processes must be 1
parallel.track(*args, output_directory=output_parent, num_processes=num_processes)

In [ ]:
analysis_options = analyze.mcs.AnalysisOptions()
analyze.mcs.process_velocities(output_parent, profile_dataset="era5_pl")
analyze.mcs.quality_control(output_parent, analysis_options)
# analyze.mcs.classify_all(output_parent, analysis_options)

In [ ]:
figure_name = f"mcs_gridrad_{event_start.replace('-', '')}"
kwargs = {"name": figure_name, "style": "presentation"}
kwargs.update({"attributes": ["velocity", "offset"]})
figure_options = option.visualize.HorizontalAttributeOptions(**kwargs)
start_time = np.datetime64(start)
end_time = np.datetime64(end)
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 7200, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)